# Проект "Flat price prediction": Извлечение датасета

In [1]:
import os
import numpy as np
import pandas as pd
import sys
import re
import time
from datetime import datetime
from bs4 import BeautifulSoup    
import requests 

In [2]:
#Используем простой звуковой индикатор для облегчения контроля за процессом
# import jupyter_beeper
# beeper = jupyter_beeper.Beeper()

In [3]:
#Также используем Progress Bar
from tqdm.notebook import tqdm

In [4]:
import ipython_exit

In [5]:
#Выставим опции pandas для удобства просмотра
pd.set_option('display.max_columns', 100)
pd.set_option('max_colwidth', 200)

In [6]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

Парсинг будем делать на локальной машине.

In [7]:
PATH_to_file = './data/'

Создадим словарь districs с наименованиями районов:

### Для парсинга сайта "krisha.kz" применим следующий подход:
- Применим фильтр по г. Алматы.  
- Применим фильтр по району и количеству комнат. В дальнейшем, возможно, по другим признакам.
- Попробуем прочесть доступное количество страниц.
- Дальше, как карта ляжет... )

Фильтр: Отдельно для каждого района и количества комнат:

In [8]:
#Верхний URL:
URL_TOP = 'https://krisha.kz/'

In [9]:
def visit_ad(f_dict_ad_params):
#Заходим в объявление    
    response = requests.get(f_dict_ad_params['url'], headers={'User-Agent': 'Mozilla/5.0'})
    if response.status_code != 200:
        #raise BaseException("response code" + str(response.status_code))
        return {}
    response.encoding = 'utf-8'
    soup = BeautifulSoup(response.text, 'html.parser')    
    
    f_dict_ad_params_full = f_dict_ad_params.copy()
    
    # Добавляем дату и время парсинга объявления
    f_dict_ad_params_full['pars_date'] = datetime.now().strftime("%d-%m-%Y %H:%M:%S")
    
    elem_offer_container = soup.find('div', class_='offer__container')
    
    offer_short_params = ['flat.building', 'flat.floor', 'live.square', 'flat.renovation', 'flat.toilet', 'flat.balcony',
                          'flat.balcony_g', 'map.complex', 'flat.door', 'inet.type', 'live.furniture', 'flat.flooring',
                          'ceiling', 'flat.security', 'has_change', 'flat.phone', 'flat.parking', 'flat.priv_dorm']
    for param in offer_short_params:
        f_dict_ad_params_full[param] = None #Инициализация
    
    #1 left bar
    elem_offer_sidebar = elem_offer_container.find('div', class_='offer__sidebar')
    elem_offer_advert_info = elem_offer_sidebar.find('div', class_='offer__advert-info')
    elem_offer_short_description = elem_offer_advert_info.find('div', class_='offer__short-description')
    #parameters ...
    elem_offer_info_items = elem_offer_short_description.find_all('div', class_='offer__info-item')#Это список параметров
    for inf_item in elem_offer_info_items[1:]:
        f_dict_ad_params_full[inf_item['data-name']] = inf_item.find('div', class_='offer__advert-short-info').text.strip()
        
    #2 right bar
    elem_offer_content = elem_offer_container.find('div', class_='offer__content')
    elem_offer_description = elem_offer_content.find('div', class_='offer__description')
    
    elem_offer_parameters = elem_offer_description.find('div', class_='offer__parameters')
    #parameters ...
    elem_offer_dls = elem_offer_parameters.find_all('dl')#Это список параметров
    for offer_dl in elem_offer_dls:
        offer_dt = offer_dl.find('dt')
        offer_dd = offer_dl.find('dd')
        f_dict_ad_params_full[offer_dt['data-name']] = offer_dd.text.strip()    
    
    # Ищем тексты объявлений
    f_dict_ad_params_full['options_text'] = None
    f_dict_ad_params_full['text'] = None    
    
    elem_offer_text = elem_offer_description.find('div', class_='text')
    #2 texts
    if elem_offer_text != None:
        #1
        elem_offer_options_text = elem_offer_text.find('div', class_='a-options-text')
        if elem_offer_options_text != None:
            f_dict_ad_params_full['options_text'] = elem_offer_options_text.text.strip()
        #2
        elem_offer_long_text = elem_offer_text.find('div', class_='a-text')
        if elem_offer_long_text != None:
            f_dict_ad_params_full['text'] = elem_offer_long_text.text.strip()
    
#     # Ищем количество просмотров
#     elem_offer_views = elem_offer_content.find('div', class_='offer__views')
#     f_dict_ad_params_full['view_count'] = str(elem_offer_views)
    
    
    #print(f_dict_ad_params_full)
    
    return f_dict_ad_params_full

In [10]:
def fill_ad_params(f_url, f_ad, f_page, f_highlighted, f_district, f_live_rooms):
    f_dict_ad_params = {}
    
    f_dict_ad_params['id'] = f_ad['data-id'] #Добавляем id
    f_dict_ad_params['page'] = f_page #Добаляем номер страницы
    f_dict_ad_params['url'] = f_url + 'a/show/' + f_ad['data-id']
    f_dict_ad_params['district'] = f_district
    f_dict_ad_params['live_rooms'] = f_live_rooms
    
    #Дальше спускаемся вниз по иерархии содержимого объявления
    elem_ad_content = f_ad.find('div', class_= 'a-card__inc')
    elem_ad_description = elem_ad_content.find('div', class_= 'a-card__descr')
    #Пропускаем 'a-card__header'
    elem_ad_main_info = elem_ad_description.find('div', class_= 'a-card__main-info')
    elem_ad_header_left = elem_ad_main_info.find('div', class_= 'a-card__header-left')
    elem_ad_title = elem_ad_header_left.find('a', class_= 'a-card__title')
    
    f_dict_ad_params['title'] = elem_ad_title.text#Заголовок объявления
    
    elem_ad_price = elem_ad_main_info.find('div', class_= 'a-card__price')
    
    f_dict_ad_params['price'] = elem_ad_price.text.strip().replace('&nbsp;', '').replace('〒', '').replace('\xa0', '')
    
    elem_ad_header_body = elem_ad_description.find('div', class_= 'a-card__header-body')
    elem_ad_subtitle = elem_ad_header_body.find('div', class_= 'a-card__subtitle')
    
    f_dict_ad_params['address'] = elem_ad_subtitle.text.strip()
    
    elem_ad_text_preview = elem_ad_header_body.find('div', class_= 'a-card__text-preview')
    
    f_dict_ad_params['text_preview'] = elem_ad_text_preview.text.strip()
    
    #Пропускаем 'a-card__footer'
    elem_ad_owner = elem_ad_description.find('div', class_= 'a-card__owner')#Owner объявления: Хозяин или Специалист
    
    if elem_ad_owner != None:# Новостройки не берем
        elem_ad_owners_label = elem_ad_owner.find('div', class_= 'owners__label')
        f_dict_ad_params['owner'] = elem_ad_owners_label.text.strip()
    else:
        return {}
   

    elem_ad_tech_info = elem_ad_description.find('div', class_= 'a-card__card-info')
    
    elem_ad_card_stats = elem_ad_tech_info.find('div', class_= 'card-stats')
    
    elem_ad_card_stats_city = elem_ad_card_stats.find('div', class_= 'card-stats__item') #Город
    f_dict_ad_params['stats_city'] = elem_ad_card_stats_city.text
    
    elem_ad_date = elem_ad_card_stats_city.find_next_sibling() #Дата объявления
    f_dict_ad_params['ad_date'] = elem_ad_date.text.strip()

#     # Ищем количество просмотров    
#     elem_ad_view_count_div = elem_ad_date.find_next_sibling()
#     elem_ad_view_count = elem_ad_view_count_div.find('span', class_ = 'a-view-count')#Количество просмотров
#     f_dict_ad_params['view_count'] = elem_ad_view_count.text
#     #print(soup.find("span", title=re.compile("RAM")).text)
    
# Значки в объявлении    
    elem_ad_paid_labels = elem_ad_tech_info.find('div', class_= 'paid-labels')    
    
    elem_ad_paid_hot = elem_ad_paid_labels.find('span', class_= 'fi-paid-hot')
    elem_ad_paid_up = elem_ad_paid_labels.find('span', class_= 'fi-paid-up')
    elem_ad_paid_fast = elem_ad_paid_labels.find('span', class_= 'fi-paid-fast')
    elem_ad_paid_urgent = elem_ad_paid_labels.find('span', class_= 'fi-paid-urgent')
    elem_ad_paid_turbo = elem_ad_paid_labels.find('span', class_= 'fi-paid-turbo')
    
    if elem_ad_paid_hot != None:
        f_dict_ad_params['paid_hot'] = 1
    else:
        f_dict_ad_params['paid_hot'] = 0
    
    if elem_ad_paid_up != None:
        f_dict_ad_params['paid_up'] = 1
    else:
        f_dict_ad_params['paid_up'] = 0
        
    if elem_ad_paid_fast != None:
        f_dict_ad_params['paid_fast'] = 1
    else:
        f_dict_ad_params['paid_fast'] = 0
        
    if elem_ad_paid_urgent != None:
        f_dict_ad_params['paid_urgent'] = 1
    else:
        f_dict_ad_params['paid_urgent'] = 0
        
    if elem_ad_paid_turbo != None:
        f_dict_ad_params['paid_turbo'] = 1
    else:
        f_dict_ad_params['paid_turbo'] = 0
        
# Помечаем выделенные объявления (которые в Highlighted Section)
    if f_highlighted:
        f_dict_ad_params['highlighted'] = 1
    else:
        f_dict_ad_params['highlighted'] = 0
    
    return f_dict_ad_params

In [11]:
def parse_by_url_district_live_rooms(f_url_top, f_district, f_live_rooms): 
    url = f'{f_url_top}prodazha/kvartiry/{f_district}/?das[live.rooms]={f_live_rooms}'

    print('Район:', f_district)
    print('Количество комнат:', round(float(f_live_rooms)))
    #Читаем страницу:
    response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
    if response.status_code != 200:
        raise BaseException("response code:" + str(response.status_code))
    response.encoding = 'utf-8'
    soup = BeautifulSoup(response.text, 'html.parser')

    #Находим главную колонку
    elem_mail_col = soup.find('div', class_='main-col')

    #Находим количество объявлений по фильтру
    elem_ads_count = elem_mail_col.find('div', class_='a-search-subtitle search-results-nb')
    ads_count = int(elem_ads_count.text.split('Найдено ', 1)[1].split(' объявлен', 1)[0].replace('\xa0', ''))
    print('Количество объявлений = ', ads_count)
    if ads_count < 1: #Если нет объявлений
        return []

    #Находим количество страниц
    elem_paginator = elem_mail_col.find('nav', class_='paginator')
    if elem_paginator != None:
        elem_paginator_btns = elem_paginator.find_all('a', class_ = 'paginator__btn')
        pages_count1 = int(elem_paginator_btns[len(elem_paginator_btns) - 2].text.strip())
    else:
        pages_count1 = 1 #Если нет переключателя страниц, то всего 1 страница

    if ads_count % 20 == 0:
        pages_count2 = int(ads_count / 20)
    else:
        pages_count2 = int(ads_count / 20) + 1

    pages_count = max(pages_count1, pages_count2)
    print(f'Количество страниц = ', pages_count1, pages_count2, pages_count)


    #Главный цикл
    flats_list_global = [] # здесь соберем список списков (эл-т - словарь параметров конкретноЙ квартиры)
    pages_list = list(range(1, pages_count + 1)) # количество страниц для парсинга
    #pages_list = list(range(1, 2 + 1)) # количество страниц для парсинга - для отладки, удалить

    actual_taken_ads = 0
    with tqdm(total=len(pages_list)) as pbar:
        for p in pages_list:
            url = f'{f_url_top}prodazha/kvartiry/{f_district}/?das[live.rooms]={f_live_rooms}&page={p}'
            response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
            if response.status_code != 200:
                print("!!!\nNetwork error. Response code: " + str(response.status_code) + '\n!!!')
                time.sleep(120)
                continue
                #raise BaseException("response code" + str(response.status_code))
            response.encoding = 'utf-8'
            soup = BeautifulSoup(response.text, 'html.parser')

            #Находим главную колонку
            elem_mail_col = soup.find('div', class_='main-col')        

            elem_main_list = elem_mail_col.find('section', class_= 'a-list a-search-list a-list-with-favs')

            #Ищем выделенную секцию
            elem_hilighted_section = elem_main_list.find('section', class_= 'highlighted-section')
            hs_ads_id = []#Список id объявлений из выделенной секции
            if elem_hilighted_section is not None:
                #Читаем объявления в выделенной секции
                elem_hilighted_section_ads = elem_hilighted_section.find_all('div', class_ = 'a-card')
                #Actually, the class is "a-card a-storage-live ddl_product ddl_product_link not-colored is-visible"
                #print('hilighted_section_ads len =', len(elem_hilighted_section_ads))
                for hs_ad in elem_hilighted_section_ads:#Для каждого объявления из выделенной секции
                    hs_ads_id.append(hs_ad['data-id'])


            #Читаем обычные объявления
            elem_common_section_ads = elem_main_list.find_all('div', class_ = 'a-card')
            #print('common_section_ads len =', len(elem_common_section_ads)) 
            
            for ad in elem_common_section_ads:#Для каждого объявления из обычной секции
                if ad['data-id'] in hs_ads_id:#Если это объявление в выделенной секции 
                    dict_ad_params = fill_ad_params(f_url_top, ad, p, True, f_district, f_live_rooms)#Заполняем словарь параметров объявления, объявление выделенное
                else:
                    dict_ad_params = fill_ad_params(f_url_top, ad, p, False, f_district, f_live_rooms)#Заполняем словарь параметров объявления, объявление НЕ выделенное

                if len(dict_ad_params) > 0:
                    time.sleep(0.5)
                    dict_ad_params_full = visit_ad(dict_ad_params)#Посещаем страницу объявления
                    if len(dict_ad_params_full) > 0:
                        flats_list_global.append(dict_ad_params_full)
                        actual_taken_ads += 1

            #Обновление ProgressBar и звуковой сигнал
            pbar.update(1)
#             if p % 10 == 0:
#                 beeper.beep(frequency=100, secs=0.3, blocking=True)
            time.sleep(2)


    print('The parsing has been done. Извлечено объявлений:', actual_taken_ads)
    return flats_list_global

In [12]:
districts = ['almaty-alatauskij', 'almaty-almalinskij', 'almaty-aujezovskij', 'almaty-bostandykskij',
             'almaty-zhetysuskij', 'almaty-medeuskij', 'almaty-nauryzbajskiy', 'almaty-turksibskij']
live_rooms = ['1','2','3','4','5.100']


# districts = ['almaty-almalinskij', 'almaty-aujezovskij', 'almaty-bostandykskij',
#              'almaty-zhetysuskij', 'almaty-medeuskij', 'almaty-nauryzbajskiy', 'almaty-turksibskij']
# live_rooms = ['1','2','3','4','5.100']

In [13]:
for distr in districts:   
    for lr in live_rooms:
        ads_list = parse_by_url_district_live_rooms(URL_TOP, distr, lr)
        if len(ads_list) > 0:
            df = pd.DataFrame(ads_list)
            display(df.head(1))
            file_date_time = datetime.now().strftime("%d-%m-%Y__%H-%M-%S")
            df.to_csv(PATH_to_file + distr + '_' + lr + '_' + file_date_time + '.csv', index=False)
        else:
            print('Таких объявлений не найдено.')

Район: almaty-alatauskij
Количество комнат: 1
Количество объявлений =  812
Количество страниц =  41 41 41



The parsing has been done. Извлечено объявлений: 808


,id,page,url,district,live_rooms,title,price,address,text_preview,owner,stats_city,ad_date,paid_hot,paid_up,paid_fast,paid_urgent,paid_turbo,highlighted,pars_date,flat.building,flat.floor,live.square,flat.renovation,flat.toilet,flat.balcony,flat.balcony_g,map.complex,flat.door,inet.type,live.furniture,flat.flooring,ceiling,flat.security,has_change,flat.phone,flat.parking,flat.priv_dorm,options_text,text
0,665875166,1,https://krisha.kz/a/show/665875166,almaty-alatauskij,1,"1-комнатная квартира, 43 м²",18500000,"Алатауский р-н, Аккент","В залоге, монолитный дом, 2010 г.п., состояние: хорошее, жил. площадь 35 кв.м., кухня 9 кв.м., потолки 2.6м., санузел совмещенный, телефон: есть возможность подключения, интернет ADSL, частично ме...",Хозяин недвижимости,Алматы,27 мар.,1,1,0,1,0,0,28-03-2021 01:02:37,"монолитный, 2010 г.п.",None,"43 м², жилая — 35 м², кухня — 9 м²",хорошее,совмещенный,лоджия,да,None,бронированная,ADSL,частично меблирована,ламинат,2.6 м,"охрана, домофон, кодовый замок, видеонаблюдение, видеодомофон",None,есть возможность подключения,рядом охраняемая стоянка,нет,"Пластиковые окна, неугловая, улучшенная, комнаты изолированы, встроенная кухня, новая сантехника, кладовка, счётчики, тихий двор.","Продам тёплую, уютную, светлую квартиру!"


Район: almaty-alatauskij
Количество комнат: 2
Количество объявлений =  734
Количество страниц =  37 37 37



The parsing has been done. Извлечено объявлений: 730


...


The parsing has been done. Извлечено объявлений: 45


,id,page,url,district,live_rooms,title,price,address,text_preview,owner,stats_city,ad_date,paid_hot,paid_up,paid_fast,paid_urgent,paid_turbo,highlighted,pars_date,flat.building,flat.floor,live.square,flat.renovation,flat.toilet,flat.balcony,flat.balcony_g,map.complex,flat.door,inet.type,live.furniture,flat.flooring,ceiling,flat.security,has_change,flat.phone,flat.parking,flat.priv_dorm,options_text,text
0,665157961,1,https://krisha.kz/a/show/665157961,almaty-turksibskij,4,"4-комнатная квартира, 91 м², 5/5 этаж",23999999,"Турксибский р-н, Паладина","кирпичный дом, 1991 г.п., состояние: среднее, санузел раздельный, частично меблирована, ❗❗❗Біз сізге жақсы аймақта жақсы пәтер ұсынамыз. Нарықтағы (қол жетімді) жылы, жайлы пәтер. Сіз пәтерді ақыс...",Проверенный специалист,Алматы,28 мар.,1,0,0,1,0,1,28-03-2021 11:40:41,"кирпичный, 1991 г.п.",5 из 5,"91 м², кухня — 12 м²",среднее,раздельный,балкон,да,None,металлическая,None,частично меблирована,None,None,"домофон, видеонаблюдение",None,None,None,нет,"Пластиковые окна, улучшенная.","❗❗❗Біз сізге жақсы аймақта жақсы пәтер ұсынамыз. \nНарықтағы (қол жетімді) жылы, жайлы пәтер. \nСіз пәтерді ақысыз және кез-келген ыңғайлы уақытта көре аласыз. \nҚоңырау шалыңыз. Асығыңыз. Сатып а..."


Район: almaty-turksibskij
Количество комнат: 5
Количество объявлений =  1
Количество страниц =  40 1 40



The parsing has been done. Извлечено объявлений: 1


,id,page,url,district,live_rooms,title,price,address,text_preview,owner,stats_city,ad_date,paid_hot,paid_up,paid_fast,paid_urgent,paid_turbo,highlighted,pars_date,flat.building,flat.floor,live.square,flat.renovation,flat.toilet,flat.balcony,flat.balcony_g,map.complex,flat.door,inet.type,live.furniture,flat.flooring,ceiling,flat.security,has_change,flat.phone,flat.parking,flat.priv_dorm,options_text,text
0,661335934,1,https://krisha.kz/a/show/661335934,almaty-turksibskij,5.100,"5-комнатная квартира, 136 м², 1/2 этаж",9520000,"Турксибский р-н, мкр Кайрат, Торгын 18","кирпичный дом, 2017 г.п., состояние: евроремонт, санузел раздельный, телефон: нет, интернет проводной, частично меблирована, Продам жилое цокольное помещение под квартиру с 5 комнат в малоэтажном ...",Специалист,Алматы,25 мар.,1,0,0,0,0,0,28-03-2021 11:41:33,"кирпичный, 2017 г.п.",1 из 2,136 м²,евроремонт,раздельный,None,None,None,None,проводной,частично меблирована,None,None,None,None,нет,паркинг,нет,None,"Продам жилое цокольное помещение под квартиру с 5 комнат в малоэтажном жилом доме. Отдельный техпаспорт на помещение. отдельный вход в квартиру. 134 кв. м., Проект был выбран как фасад частного до..."


In [14]:
ipython_exit.exit()

IpyExit: 

### В итоге мы получили тренировочный датасет train.csv с xxxx записей